In [1]:
import itertools
from sklearn import preprocessing
from random import randint
import numpy as np
from sklearn import preprocessing
from sklearn.svm import LinearSVC
import random

In [2]:
pairs = []
for l in itertools.combinations(range(5), 2):
    pairs.append(l)
print pairs

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [3]:
def train_test_split(indices):
    og_data = np.load('leaveoneout/data_p1.npy')
    labels = np.load('leaveoneout/labels_p1.npy')
    conds = np.load('leaveoneout/conds_p1.npy')
    
    # Label encoder
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels = le.transform(labels)
    x_train, x_test, y_train, y_test = [], [], [], []

    smin = 0
    smax = 1
    x = og_data
    xmax, xmin = x.max(), x.min()
    norm_data = (x - xmin)*(smax - smin)/(xmax - xmin) + smin

    # Sorting the data
    sorted_index = np.argsort(labels)
    sorted_labels = labels[sorted_index]
    sorted_og_data = og_data[sorted_index]
    sorted_conds = conds[sorted_index]
    sorted_norm_data = norm_data[sorted_index]

    # split the data into train and test
#     test_random_indices = []
#     for i in range(sorted_labels.shape[0]/split_part):
#         index = i*split_part + randint(0, split_part - 1)
#         test_random_indices.append(index)

    x_train = np.delete(sorted_norm_data, indices, 0)
    x_test = sorted_norm_data[indices]
    y_train = np.delete(sorted_labels, indices)
    y_test = sorted_labels[indices]
    z_train = np.delete(sorted_conds, indices)
    z_test = sorted_conds[indices]
    
    return x_train, x_test, y_train, y_test, z_train, z_test


def get_mat(x_train):
    voxel_corr_matrix = []
    size = x_train.shape[0]/5
    for i in range(x_train.shape[1]):
        voxel = x_train[:, i]
        pre_corr_matrix = []
        for j in range(size):
            k = j*5
            pre_corr_matrix.append(voxel[k:k+5])

        voxel_corr_matrix.append(np.transpose(pre_corr_matrix))

    voxel_corr_matrix = np.asarray(voxel_corr_matrix)
    
    return voxel_corr_matrix

In [39]:
seq = np.load('seq.npy')

acc = []
for i in range(6):
    
    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(seq[:, i])

    voxel_corr_matrix = get_mat(x_train)

    corr_arr = []
    for i in range(voxel_corr_matrix.shape[0]):
        voxel = voxel_corr_matrix[i]
        corr = 0.0
        for pair in pairs:
            corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
        corr_arr.append(corr/len(pairs))

    c_a = np.asarray(corr_arr)
    sorted_index = np.argsort(c_a)[::-1]

    x_t = x_train[:, sorted_index[:800]]
    x_ts = x_test[:, sorted_index[:800]]

    clf = LinearSVC()

    clf.fit(x_t, y_train)
    
    results_top = []
    for i in [1, 3, 5, 10]:
        predicted_correct = 0
        for k in range(y_test.shape[0]):
            o = x_ts[k]
            o = o.reshape(1, -1)
            decision_lbl = clf.decision_function(o)[0]
            top_class = i
            top_n = np.argsort(decision_lbl)[::-1][:top_class]
            if y_test[k] in top_n:
                predicted_correct += 1.0/60
        results_top.append(predicted_correct)
    acc.append(results_top)

60
60
60
60
60
60


In [40]:
print np.average(acc, axis=0)
print np.std(acc, axis=0)

[0.51111111 0.74166667 0.83611111 0.92222222]
[0.05061352 0.05159493 0.04453117 0.0248452 ]


In [5]:
conds = np.load('leaveoneout/conds_p1.npy')
conds = np.unique(conds)
print conds

['animal' 'bodypart' 'building' 'buildpart' 'clothing' 'furniture'
 'insect' 'kitchen' 'manmade' 'tool' 'vegetable' 'vehicle']


In [77]:
seq = np.load('seq.npy')
conds = np.load('leaveoneout/conds_p1.npy')
conds = np.unique(conds)

loo_acc = []
loo_std = []
for c in conds:
    acc = []
    for i in range(6):
        x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(seq[:, i])

        indices = [i for i, x in enumerate(z_train) if x == c]

        corr_train = np.delete(x_train, indices, 0)

        voxel_corr_matrix = get_mat(corr_train)

#         print voxel_corr_matrix.shape

        corr_arr = []
        for i in range(voxel_corr_matrix.shape[0]):
            voxel = voxel_corr_matrix[i]
            corr = 0.0
            for pair in pairs:
                corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
            corr_arr.append(corr/len(pairs))

        c_a = np.asarray(corr_arr)
        sorted_index = np.argsort(c_a)[::-1]

        x_t = x_train[:, sorted_index[:800]]
        x_ts = x_test[:, sorted_index[:800]]

        clf = LinearSVC()

        clf.fit(x_t, y_train)
        results_top = []
        for i in [1, 3, 5, 10]:
            predicted_correct = 0
            for k in range(y_test.shape[0]):
                o = x_ts[k]
                o = o.reshape(1, -1)
                decision_lbl = clf.decision_function(o)[0]
                top_class = i
                top_n = np.argsort(decision_lbl)[::-1][:top_class]
                if y_test[k] in top_n:
                    predicted_correct += 1.0/60
            results_top.append(predicted_correct)
        acc.append(results_top)
    loo_acc.append(np.average(acc, axis=0))
    loo_std.append(np.std(acc, axis=0))
    
    print loo_acc

[array([0.51944444, 0.725     , 0.83888889, 0.91111111])]
[array([0.51944444, 0.725     , 0.83888889, 0.91111111]), array([0.525     , 0.72222222, 0.81944444, 0.91388889])]
[array([0.51944444, 0.725     , 0.83888889, 0.91111111]), array([0.525     , 0.72222222, 0.81944444, 0.91388889]), array([0.50833333, 0.72777778, 0.81666667, 0.91944444])]
[array([0.51944444, 0.725     , 0.83888889, 0.91111111]), array([0.525     , 0.72222222, 0.81944444, 0.91388889]), array([0.50833333, 0.72777778, 0.81666667, 0.91944444]), array([0.49722222, 0.73055556, 0.83888889, 0.91111111])]
[array([0.51944444, 0.725     , 0.83888889, 0.91111111]), array([0.525     , 0.72222222, 0.81944444, 0.91388889]), array([0.50833333, 0.72777778, 0.81666667, 0.91944444]), array([0.49722222, 0.73055556, 0.83888889, 0.91111111]), array([0.51666667, 0.70555556, 0.80555556, 0.92222222])]
[array([0.51944444, 0.725     , 0.83888889, 0.91111111]), array([0.525     , 0.72222222, 0.81944444, 0.91388889]), array([0.50833333, 0.7277

In [78]:
print np.average(loo_acc, axis=0)
print np.std(loo_acc, axis=0)

[0.51111111 0.72800926 0.82662037 0.91921296]
[0.00928238 0.00874436 0.00998058 0.00572186]


In [79]:
shuffle = np.load('shuffle.npy')
conds = np.load('leaveoneout/conds_p1.npy')
conds = np.unique(conds)

loo_acc = []
loo_std = []
for c in conds:
    acc = []
    for i in range(6):
        x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(shuffle[:, i])

        indices = [i for i, x in enumerate(z_train) if x == c]

        corr_train = np.delete(x_train, indices, 0)

        voxel_corr_matrix = get_mat(corr_train)

#         print voxel_corr_matrix.shape

        corr_arr = []
        for i in range(voxel_corr_matrix.shape[0]):
            voxel = voxel_corr_matrix[i]
            corr = 0.0
            for pair in pairs:
                corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
            corr_arr.append(corr/len(pairs))

        c_a = np.asarray(corr_arr)
        sorted_index = np.argsort(c_a)[::-1]

        x_t = x_train[:, sorted_index[:800]]
        x_ts = x_test[:, sorted_index[:800]]

        clf = LinearSVC()

        clf.fit(x_t, y_train)
        results_top = []
        for i in [1, 3, 5, 10]:
            predicted_correct = 0
            for k in range(y_test.shape[0]):
                o = x_ts[k]
                o = o.reshape(1, -1)
                decision_lbl = clf.decision_function(o)[0]
                top_class = i
                top_n = np.argsort(decision_lbl)[::-1][:top_class]
                if y_test[k] in top_n:
                    predicted_correct += 1.0/60
            results_top.append(predicted_correct)
        acc.append(results_top)
    loo_acc.append(np.average(acc, axis=0))
    loo_std.append(np.std(acc, axis=0))
    
    print loo_acc

[array([0.51944444, 0.72777778, 0.82777778, 0.91666667])]
[array([0.51944444, 0.72777778, 0.82777778, 0.91666667]), array([0.525     , 0.72222222, 0.81666667, 0.91111111])]
[array([0.51944444, 0.72777778, 0.82777778, 0.91666667]), array([0.525     , 0.72222222, 0.81666667, 0.91111111]), array([0.49444444, 0.73055556, 0.81944444, 0.92222222])]
[array([0.51944444, 0.72777778, 0.82777778, 0.91666667]), array([0.525     , 0.72222222, 0.81666667, 0.91111111]), array([0.49444444, 0.73055556, 0.81944444, 0.92222222]), array([0.51111111, 0.73055556, 0.83055556, 0.90833333])]
[array([0.51944444, 0.72777778, 0.82777778, 0.91666667]), array([0.525     , 0.72222222, 0.81666667, 0.91111111]), array([0.49444444, 0.73055556, 0.81944444, 0.92222222]), array([0.51111111, 0.73055556, 0.83055556, 0.90833333]), array([0.49444444, 0.70833333, 0.81388889, 0.91388889])]
[array([0.51944444, 0.72777778, 0.82777778, 0.91666667]), array([0.525     , 0.72222222, 0.81666667, 0.91111111]), array([0.49444444, 0.7305

In [80]:
print np.average(loo_acc, axis=0)
print np.std(loo_acc, axis=0)

[0.52037037 0.7287037  0.825      0.91712963]
[0.01560398 0.00971119 0.00699059 0.00492136]


In [49]:
shuffle_indices = []
for i in range(60):
    index = np.array([0, 1, 2, 3, 4])
    random.shuffle(index)
    shuffle_indices.append(index + 5*i)

In [57]:
si = np.asarray(shuffle_indices)

In [65]:
si[:, 0].shape

(60,)

In [61]:
x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(shuffle[:, 0])

In [67]:
corr_train = x_train[si[:, 0]]

print corr_train.shape

(60, 21764)


In [4]:
voxel_corr_matrix = get_mat(corr_train)

voxel_corr_matrix.shape

corr_arr = []
for i in range(voxel_corr_matrix.shape[0]):
    voxel = voxel_corr_matrix[i]
    corr = 0.0
    for pair in pairs:
        corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
    corr_arr.append(corr/len(pairs))

c_a = np.asarray(corr_arr)
sorted_index = np.argsort(c_a)[::-1]

NameError: name 'corr_train' is not defined

In [69]:
    corr_arr = []
    for i in range(voxel_corr_matrix.shape[0]):
        voxel = voxel_corr_matrix[i]
        corr = 0.0
        for pair in pairs:
            corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
        corr_arr.append(corr/len(pairs))

    c_a = np.asarray(corr_arr)
    sorted_index = np.argsort(c_a)[::-1]

    x_t = x_train[:, sorted_index[:800]]
    x_ts = x_test[:, sorted_index[:800]]

    clf = LinearSVC()

    clf.fit(x_t, y_train)
    results_top = []
    for i in [1, 3, 5, 10]:
        predicted_correct = 0
        for k in range(y_test.shape[0]):
            o = x_ts[k]
            o = o.reshape(1, -1)
            decision_lbl = clf.decision_function(o)[0]
            top_class = i
            top_n = np.argsort(decision_lbl)[::-1][:top_class]
            if y_test[k] in top_n:
                predicted_correct += 1.0/60
        results_top.append(predicted_correct)
    print results_top

KeyboardInterrupt: 

In [113]:
seq = np.load('shuffle.npy')
conds = np.load('leaveoneout/conds_p1.npy')
conds = np.unique(conds)

acc = []
for i in range(6):
    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(seq[:, i])

    rand_smpl = [conds[i] for i in random.sample(xrange(len(conds)), 1)]

    indices = []
    for r in rand_smpl:
        indices += [i for i, x in enumerate(z_train) if x == r]

    print len(indices)
    
    corr_train = x_train[indices]
    voxel_corr_matrix = get_mat(corr_train)

#         print voxel_corr_matrix.shape

    corr_arr = []
    for i in range(voxel_corr_matrix.shape[0]):
        voxel = voxel_corr_matrix[i]
        corr = 0.0
        for pair in pairs:
            corr += np.corrcoef(voxel[pair[0]], voxel[pair[1]])[0, 1]
        corr_arr.append(corr/len(pairs))

    c_a = np.asarray(corr_arr)
    sorted_index = np.argsort(c_a)[::-1]

    x_t = x_train[:, sorted_index[:800]]
    x_ts = x_test[:, sorted_index[:800]]

    clf = LinearSVC()

    clf.fit(x_t, y_train)
    results_top = []
    for i in [1, 3, 5, 10]:
        predicted_correct = 0
        for k in range(y_test.shape[0]):
            o = x_ts[k]
            o = o.reshape(1, -1)
            decision_lbl = clf.decision_function(o)[0]
            top_class = i
            top_n = np.argsort(decision_lbl)[::-1][:top_class]
            if y_test[k] in top_n:
                predicted_correct += 1.0/60
        results_top.append(predicted_correct)
    acc.append(results_top)

25
25
25
25
25
25


In [114]:
print np.average(acc, axis=0)
print np.std(acc, axis=0)

[0.26666667 0.45555556 0.59166667 0.73055556]
[0.05270463 0.01571348 0.0356812  0.0310565 ]


In [85]:
conds = np.load('leaveoneout/conds_p1.npy')
conds = np.unique(conds)

In [97]:
rand_smpl = [conds[i] for i in random.sample(xrange(len(conds)), 4)]
print rand_smpl

['animal', 'clothing', 'bodypart', 'building']


In [98]:
x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(seq[:, 0])

In [101]:
indices = []
for r in rand_smpl:
    indices += [i for i, x in enumerate(z_train) if x == r]

In [105]:
indices = sorted(indices)
print len(indices)

100
